In [1]:
#LIBRARIES
import pandas as pd
import re
import numpy as np
import math

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## nomalice parameters
from unidecode import unidecode

import spacy  # For preprocessing
from scipy import spatial
from time import time  # To time our operations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

## user to tranlate words

In [2]:

from deep_translator import GoogleTranslator
def translateString(to_translate, languageFinal):
    translated = GoogleTranslator(source='auto', target=languageFinal).translate(to_translate)
    return translated
def tranlateColum(dataFrame,columName,languageFinal):
    ## dataFrame Lengt
    total = dataFrame.shape[0]
    for i in range(0 ,total):
        to_translate = dataFrame[columName][i]
        translated = translateString(to_translate,languageFinal)
        print(i,translated)
        dataFrame[columName][i] = translated
    

## nfinal data base

In [ ]:

df = pd.read_excel("2_final_db.xlsx")


## se eliminan las perosonas sin job title 

In [ ]:

dfa = df[df["job_title"].notna()] 
dfa = dfa.reset_index(drop = True) 
dfa.shape[0]
dfa["job_title"][0]

## se traduce al ingles todo y se guarda en su propio archivo

In [ ]:

tranlateColum(dfa,"job_title","en")
dfa["job_title"][200]
dfa.to_excel("job_title_en.xlsx")

In [ ]:
# cuales del la db original tienen valoes nulos, por columna

df.isnull().sum()

# Se carga los job titles en ingles 

In [3]:
EN_DF = pd.read_excel("job_title_en.xlsx")
EN_DF.shape

(399, 38)

## Cleaning and lemmatization of the job titles

In [4]:
nlp = spacy.load("en_core_web_sm")

def cleaning(text):
    ##text = text.lower()
    #text = text.replace('[>/,\.!?\-!?\n\)\(\r]', ' ') 
    text = re.sub("[^A-Za-z']+", ' ', str(text))
    doc = nlp(text)
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    #txt = [token.lemma_ for token in doc if not token.is_stop]
    txt = [tok.lemma_.lower() for tok in doc
                if not tok.is_stop and not tok.is_punct and tok.is_alpha ]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    return txt


## loading word vectors+
 ## TO A python dic

In [5]:

embeddings_dict = {}
# vectors dimention
DIMENTION_DIC = 300
# 6 billions words
with open("glove.6B."+str(DIMENTION_DIC)+"d.txt", 'r', encoding="utf-8") as f:
# 42 Billion words
#with open("glove.42B."+str(DIMENTION_DIC)+"d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        print(word)
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

rmium
-4.1
merkez
briareus
helst
917k
levantines
record-low
:2009
half-japanese
sistrum
berlei
electronvolts
angustia
crispum
hyperacute
meq/l
bavarii
mekilta
abct
lowest-ranking
rupertswood
undulatory
subdiaconate
wilbrand
acrocorinth
animality
time-scales
oralism
imino
bortezomib
panteion
chalkida
dipavamsa
vibia
kues
ariès
milà
nonassociative
satyrus
iambs
ghazis
demon-like
hydroxyls
s-iv
anorthite
jahandar
step-grandmother
dmitrievna
al-basri
masmuda
amazonians
baasha
4-volume
suhi
triumphales
redocking
purinergic
three-second
allomorphs
stinkwood
adefonsi
höger
low-down
gravimeter
hit-and-miss
kaap
porajmos
foreshadowings
reconsiderations
xanthippus
alectryon
adfs
malatia
vasorum
i-mutation
all-professional
52,000-seat
ahcc
segmentata
reattaches
self-justification
nisba
farhud
narek
milanesa
silooy
adam-ondi-ahman
coniine
teruko
3dlabs
esbjörn
shw
tauberbischofsheim
58-60
electrodeposition
lab-on-a-chip
impregnability
amoebiasis
2339
process-based
univoltine
myrmecia
gortyna
43c
s

In [6]:
## ordena la spalabras dependiendo de la distancia euclidiana en el n-space
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

## method to get word/vector from dictionary
def getVectorWord(dictionary, word):
    if word in dictionary:
        return dictionary[word]
    else :
        return np.zeros(DIMENTION_DIC)


## se buscan las palabras mas cercanas 
### manager
### developer
### artist

In [ ]:
print(find_closest_embeddings(embeddings_dict["manager"])[:10])
print(find_closest_embeddings(embeddings_dict["developer"])[:10])
print(find_closest_embeddings(embeddings_dict["artist"])[:10])

## word math, sumando vectores
### software + developer + ui
### software + developer + art
### software + developer - ui

In [ ]:

print(find_closest_embeddings(embeddings_dict["software"] + embeddings_dict["developer"]+ embeddings_dict["ui"])[:7])
print(find_closest_embeddings(embeddings_dict["software"] + embeddings_dict["developer"]+ embeddings_dict["art"])[:7])
print(find_closest_embeddings(embeddings_dict["software"] + embeddings_dict["developer"]- embeddings_dict["ui"])[:7])

In [ ]:
print(find_closest_embeddings(embeddings_dict["hand"] + embeddings_dict["finger"]- embeddings_dict["ring"])[:7])

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
words =  list(["developer","front-end","sales","art","politics","math","calculator"])
vectors = [embeddings_dict[word] for word in words]

In [ ]:
Y = tsne.fit_transform(vectors[:100])
plt.scatter(Y[:, 0], Y[:, 1])
for label, x, y in zip(words, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords="offset points")
plt.show()

In [7]:
## tranform profiles to vectors
profilesVectors = []
for index in range(0,len(EN_DF['job_title'])):
    row = EN_DF['job_title'][index]
    cleaned = cleaning(row)
    nVectors = [np.zeros(DIMENTION_DIC)]
    if len(cleaned) > 0:
        nVectors  = [ getVectorWord(embeddings_dict,word) for word in cleaned]
    
        
    #print(len(nVectors),len(cleaned))
    sumVector = np.add.reduce(nVectors)
   
            
    #print(type(sumVector))
    profilesVectors.append(sumVector)

In [ ]:
len(profilesVectors)

In [ ]:
len(EN_DF['job_title'])


In [ ]:
print(find_closest_embeddings(profilesVectors[200])[1:9])
print(EN_DF.loc[200])

In [ ]:
print(find_closest_embeddings(profilesVectors[197])[1:9])
print(EN_DF.loc[197])


In [ ]:
## seleccionando el numero de categorias a mano
TOTAL_CATEGORIES = 10
kmeans = KMeans(n_clusters=TOTAL_CATEGORIES, random_state=0).fit(profilesVectors)

In [ ]:
for i in range(0,len(kmeans.cluster_centers_)):
    print(find_closest_embeddings(kmeans.cluster_centers_[i])[:5])

In [ ]:
len(embeddings_dict.keys())

In [ ]:
kmeans.labels_

In [ ]:
len(kmeans.labels_)

## routine to generate external file, to assing the relation OR/NOT to computer


In [8]:
## number of words needed to pass
MIN_COUNT = 2
## validates if is word related
def validator(relatedWords, words):
    isRelated = False
    count = 0
    for word in words:
        if word in relatedWords:
            count+=1
        if count >= MIN_COUNT: 
            return True
    if count >= MIN_COUNT: 
        return True
    else:
        return False

 



In [9]:
TOTAL_CLUSTERS = 12
dicC = {}
clusterTotal = np.zeros(len(EN_DF['job_title']))

related_words = ["software","development","engineer","web","developers","developer","engineering","technical","java","computer","microsoft","technology","programmer"]
for clust in range(2, TOTAL_CLUSTERS+1):
    kmeans = KMeans(n_clusters=clust, random_state=0).fit(profilesVectors)
    labels = kmeans.labels_
    relatedArray = []
    for index in range(0,len(kmeans.cluster_centers_)):
        words = find_closest_embeddings(kmeans.cluster_centers_[index])[:7]
        related = validator(related_words, words)
        print(index,related," - ",words)
        relatedArray.append(related)
    print(len(relatedArray),clust, len(clusterTotal))
    for proIndex in range(0,len(labels)):
        label = labels[proIndex]
        isRelated = relatedArray[label]
        if isRelated==True:
            print("isRelated")
            prev = clusterTotal[proIndex]
            clusterTotal[proIndex] = prev + 1
            print(prev,clusterTotal[proIndex])
    print("------------------------------------------\n")

d
3.0 4.0
isRelated
3.0 4.0
isRelated
0.0 1.0
isRelated
2.0 3.0
isRelated
0.0 1.0
isRelated
3.0 4.0
isRelated
3.0 4.0
isRelated
3.0 4.0
isRelated
1.0 2.0
isRelated
0.0 1.0
isRelated
3.0 4.0
isRelated
2.0 3.0
isRelated
3.0 4.0
isRelated
3.0 4.0
isRelated
4.0 5.0
------------------------------------------

0 True  -  ['management', 'technology', 'business', 'services', 'research', 'development', 'information']
1 False  -  ['director', 'manager', 'management', 'marketing', 'said', 'business', 'executive']
2 True  -  ['engineer', 'software', 'computer', 'engineering', 'engineers', 'technology', 'systems']
3 True  -  ['developer', 'developers', 'software', 'web', 'application', 'hardware', 'user']
4 False  -  ['actress', 'actor', 'singer', 'starred', 'actresses', 'starring', 'musician']
5 False  -  ['_____________________________________________', 'bulletinyyy', 'str95bb', 'js94bb', 'k587-1', 'bdb94', 'bb96']
6 False  -  ['designer', 'designers', 'design', 'graphic', 'designs', 'fashion', '

In [10]:
dta = {}
dta["total_clustering"]=clusterTotal
JOB_CLUS_DF = pd.DataFrame(data=dta)
print(JOB_CLUS_DF, JOB_CLUS_DF.shape)

     total_clustering
0                 0.0
1                 0.0
2                 0.0
3                 0.0
4                11.0
..                ...
394               0.0
395               9.0
396              10.0
397               0.0
398              11.0

[399 rows x 1 columns] (399, 1)


In [11]:
JOB_CLUS_DF.to_excel("index-by-cluster.xlsx",index=False)

In [ ]:
EN_DF